## Exercise 2 (book p.63)

(a) This is a regression problem, because the dependent variable CEO salary is a continous (and not a classification) variable. Here we're more interested in inference. We have n=500, and p=3.

(b) This is a classification problem because the dependent variable, the outcome of product launch, is binary. In this case we're more interested in prediction. We have n=20, and p=13.

(c) This is a regression problem, since the dependent variable (% change in the USD/Euro exchange rate) is continous. Once again here we're more interested in prediction. We have n=52, and p=3.